# 1. Logistic Regression

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_file_1 = Path(os.path.abspath("")) / "data" / "ex2data1.txt"
data_set_1 = pd.read_csv(data_file_1, header=None, names=["Score_1", "Score_2", "Admitted"])
# print(data_set_1)
# print(data_set_1.head())
# print(data_set_1.describe())

x = np.matrix(data_set_1.iloc[:, 0:2])
y = np.matrix(data_set_1.iloc[:, 2:])
print(x)
print(y)

## 1.1 Visualizing the data

In [ ]:
# init empty array with dimensions
admitted_list = np.empty((0, 3))
non_admitted_list = np.empty((0, 3))

for record in np.array(data_set_1):
    if record[2] == 1:
        admitted_list = np.append(admitted_list, [record], axis=0)
    else:
        non_admitted_list = np.append(not_admitted_list, [record], axis=0)

# simple way to init
# admitted_list = np.array([record for record in np.array(data_set_1) if record[2] == 1])
# not_admitted_list = np.array([record for record in np.array(data_set_1) if record[2] == 0])

# print(admitted_list)
# print(not_admitted_list)

plt.plot(admitted_list[:, 0], admitted_list[:, 1], 'go', label="Admitted")
plt.plot(non_admitted_list[:, 0], non_admitted_list[:, 1], 'ro', label="Non-admitted")
plt.xlabel("Score_1")
plt.ylabel("Score_2")
plt.legend(loc="upper right")
plt.show()
